In [1]:
import numpy as np
import json
import ast
import os
import glob
import shutil

from honeybee.hbsurface import HBSurface, HBAnalysisSurface
from honeybee.radiance.analysisgrid import AnalysisGrid
from honeybee.radiance.recipe.daylightcoeff.gridbased import DaylightCoeffGridBased
from honeybee.radiance.sky.skymatrix import SkyMatrix as skm

from ladybug.wea import Wea
from ladybug.sunpath import Sunpath
from ladybug.analysisperiod import AnalysisPeriod

from IPython.display import display, clear_output

Failed to import projection factor splines from CSV.
A simpler interoplation method for Solarcal will be used.
Failed to import projection factor splines from CSV.
A simpler interoplation method for Solarcal will be used.
Successfully imported Ladybug plugin: ladybug_comfort
Successfully imported Ladybug plugin: ladybug_geometry
Failed to load paths from C:\Users\user\Miniconda3\lib\site-packages\honeybee\config.json.
Path to OpenStudio is set to: C:/Program Files/OpenStudio 1.14.0
Path to radiance is set to: c:/radiance
Path to perl is set to: C:/Program Files/OpenStudio 1.14.0\strawberry-perl-5.16.2.1-32bit-portable-reduced


In [5]:
hoys = np.load('./data/hoys.npy')
hoys = list(hoys)
len(hoys)

132

In [6]:
#Set up epw, analysis period and climate-based sky matrix

epw_file_path = './data/DNK_Copenhagen.061800_IWEC.epw'

wea = Wea.from_epw_file(epw_file_path)

#period = AnalysisPeriod(st_month=1, st_day=15, st_hour=8, end_month=12, end_day=15, end_hour=18)

sky = skm(wea, sky_density=1, hoys=hoys)

In [7]:
#Function to generate hb_surface out of the json file

def get_hb_surfaces(file):
    hb_surfaces = []
    with open(file, 'r') as in_file:
        data = json.load(in_file)
        for srf in data:
            srf = ast.literal_eval(srf)
            
            hb_surface = HBSurface.from_json(srf)
            #Append to the surfaces list
            hb_surfaces.append(hb_surface)
            
    print("{} Honeybee Surfaces parsed".format(len(hb_surfaces)))
    
    return hb_surfaces

In [8]:
#Function to generate an analysis grid out of the json file
def get_hb_grid(file):
    
    with open(file, 'r') as in_file:
        data = json.load(in_file)
        grid = [AnalysisGrid.from_json(data)]
    return grid

In [9]:
# File locations and save folders

#Folder containing hb surface json files
surface_folder = r'F:/ML/CaseStudies/IndoorGAN/JSONs/surfaces/Running'

#Folder containing grid json files
grid_folder = r'F:\ML\CaseStudies\IndoorGAN\JSONs\grids/Running'


#Folders to save daylight results
da_folder = r'F:\ML\CaseStudies\IndoorGAN\JSONs\results\DA'
cda_folder = r'F:\ML\CaseStudies\IndoorGAN\JSONs\results\CDA'
udi_folder = r'F:\ML\CaseStudies\IndoorGAN\JSONs\results\UDI'
udi_less_folder = r'F:\ML\CaseStudies\IndoorGAN\JSONs\results\UDI_less'
udi_more_folder = r'F:\ML\CaseStudies\IndoorGAN\JSONs\results\UDI_more'

#Folder to save recipes
target_folder = os.path.join(os.getcwd(), 'recipes')

In [10]:
#Get json files out of the locations

json_surfaces = glob.glob(surface_folder + '\*.json')
json_grids = glob.glob(grid_folder + '\*.json')

assert len(json_surfaces) == len(json_grids)

len(json_surfaces), len(json_grids)

(0, 0)

In [29]:
def run_batch_simulation(surfaces, grids):
    i=0
    #Loop over all cases / json files
    for srf, grid in zip(surfaces, grids):
        clear_output(wait=True)
        i+=1
        print("Now running case {}...".format(int(i)))
        
        #Create hb surfaces
        hb_surfaces = get_hb_surfaces(srf)
        #Create analysis grid
        grid = get_hb_grid(grid)
        print('Daylight simulation running for {} total grid points'.format(len(grid[0])))
        
        #Create recipe
        recipe = DaylightCoeffGridBased(sky_mtx=sky, analysis_grids=grid, hb_objects=hb_surfaces)
        
        #Run annual simulation
        print('\n')
        command_path = recipe.write(target_folder, 'current')
        recipe.run(command_path)
        print('\n')
        #Calculate result grid
        result = recipe.results()[0]
        print('Calculating annual metrics..')
        #Calculate annual metrics
        da, cda, udi, udi_less, udi_more = result.annual_metrics()
        print('\n')
        print('Saving results to disk..')
        #Save metrics into arrays
        np.save(da_folder + '\\' + srf.split('\\')[-1].split('.json')[0], np.asarray(da))
        np.save(cda_folder + '\\' + srf.split('\\')[-1].split('.json')[0], np.asarray(cda))
        np.save(udi_folder + '\\' + srf.split('\\')[-1].split('.json')[0], np.asarray(udi))
        np.save(udi_less_folder + '\\' + srf.split('\\')[-1].split('.json')[0], np.asarray(udi_less))
        np.save(udi_more_folder + '\\' + srf.split('\\')[-1].split('.json')[0], np.asarray(udi_more))
        
        shutil.rmtree(target_folder + '\current', ignore_errors=True)

In [30]:
run_batch_simulation(json_surfaces, json_grids)

Now running case 50...
696 Honeybee Surfaces parsed
Daylight simulation running for 9615 total grid points
Found 476 opaque surfaces.
Found 220 fenestration surfaces.
Found 0 window-groups.


Writing recipe contents to: F:\ML\CaseStudies\IndoorGAN\JSONs\recipes\current\gridbased_daylightcoeff
Calculating solar values...
# Number of sun up hours: 120
Writing sun matrix to F:\ML\CaseStudies\IndoorGAN\JSONs\recipes\current\gridbased_daylightcoeff\sky\sunmtx.smx


Unloading the current values from the analysis grids.

loading total and direct results for Floor AnalysisGrid from scene::default
F:\ML\CaseStudies\IndoorGAN\JSONs\recipes\current\gridbased_daylightcoeff\result/scene..default.ill
F:\ML\CaseStudies\IndoorGAN\JSONs\recipes\current\gridbased_daylightcoeff\result\sun..scene..default.ill

Calculating annual metrics..


Saving results to disk..
